# Lab 3: Text clustering

Generate and interpret document and term clusters from hotel reviews

Objectives:

- work with sklearn components
- construct document-term and term-document matrices
- apply dimensionality reduction
- perform cluster analysis on document and term collections
- interpret results

In [ ]:
import re

import nltk
import numpy as np
import pandas as pd
from cytoolz import *

In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
df = pd.read_pickle("/data/hotels_id.pkl")
lax = df.query('offering_id==1762573')

In [ ]:
df.head()

,title,text,overall,value,service,cleanliness,offering_id
0,"“Truly is ""Jewel of the Upper Wets Side""”","Stayed in a king suite for 11 nights and yes it cots us a bit but we were happy with the standard of room, the location and the friendliness of the staff. Our room was on the 20th floor overlooking Broadway and the madhouse of the Fairway Market. Room was quite with no noise evident from the hallway or adjoining rooms. It was great to be able to open windows when we craved fresh rather than heated air. The beds, including the fold out sofa bed, were comfortable and the rooms were cleaned well. Wi-fi access worked like a dream with only one connectivity issue on our first night and this was promptly responded to with a call from the service provider to ensure that all was well. The location close to the 72nd Street subway station is great and the complimentary umbrellas on the drizzly days were greatly appreciated. It is fabulous to have the kitchen with cooking facilities and the access to a whole range of fresh foods directly across the road at Fairway.\nThis is the second time that members of the party have stayed at the Beacon and it will certainly be our hotel of choice for future visits.",5.0,5.0,5.0,5.0,93338
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the place we love to stay. So conveniently located to Central Park, Lincoln Center and great local restaurants. The rooms are lovely - beds so comfortable, a great little kitchen and new wizz bang coffee maker. The staff are so accommodating and just love walking across the street to the Fairway supermarket with every imaginable goodies to eat (if you choose not to go out for every meal!)",5.0,5.0,5.0,5.0,93338
2,“Great Stay”,"This is a great property in Midtown. We two different rooms different rooms during our stay. The first room was in the North tower, which was quite inconvenient. You have to go through the conference area to get to the north elevators. \nThe second room was the Andaz Suite. It was nicely appointed room, but the best part about it was the bathroom. From the foot soaking bowl to the bath products, everything about the bathroom was awesome!\nLemon poppy-seed pancakes are must haves at the restaurant. One of the best pancakes ever.",4.0,4.0,4.0,5.0,1762573
3,“Modern Convenience”,"The Andaz is a nice hotel in a central location of Manhattan. The Hyatt has come up with a modern hotel that is both comfortable and convenient. When you arrive you are greeted by friendly ""Hosts"" and they walk you to the check-in desk while offering you a beverage. \nWe had a one bedroom suite that accommodated four people reasonably well. Plenty of closet space, well lit with floor to ceiling windows, and actually quiet!\nThe bathroom was large with a very nice walk-in shower and a built-in bench with unique low spout to wash your feet.\nThe kitchenette was a nice touch with a stocked fridge offering complimentary non-alcoholic beverages and snacks, dishes and utensils, a sink, dishwasher, and a microwave. \nThey have daily Happy Hour(s) where you can get a complimentary decent glass of wine in the modest Lobby Lounge and bring it to your room. The Lobby Lounge has some seating and one table with 8 or so chairs where you can buy food from the adjacent restaurant. One suggestion is to offer selections of cheese and crackers platters to go with that wine. We ordered one that had to be custom made, but it worked well. \nWe didn't eat in the hotel restaurants. The restaurant off the lobby was very pricey for breakfast (+$20 per entree). When you can get a decent breakfast within a block or two of the hotel for under $10, what can I say?\nAs a hotel designer and someone who is very picky about hotels, I will definitely stay there again and I highly recommend it.",4.0,5.0,5.0,5.0,1762573
4,“Its the best of the Andaz Brand in the US....”,"I have stayed at each of the US Andaz properties, and this one is the best of them all. MUCH better than the Wes

In [ ]:
len(lax)

636

In [ ]:
lax.head()

,title,text,overall,value,service,cleanliness,offering_id
2,“Great Stay”,"This is a great property in Midtown. We two different rooms different rooms during our stay. The first room was in the North tower, which was quite inconvenient. You have to go through the conference area to get to the north elevators. \nThe second room was the Andaz Suite. It was nicely appointed room, but the best part about it was the bathroom. From the foot soaking bowl to the bath products, everything about the bathroom was awesome!\nLemon poppy-seed pancakes are must haves at the restaurant. One of the best pancakes ever.",4.0,4.0,4.0,5.0,1762573
3,“Modern Convenience”,"The Andaz is a nice hotel in a central location of Manhattan. The Hyatt has come up with a modern hotel that is both comfortable and convenient. When you arrive you are greeted by friendly ""Hosts"" and they walk you to the check-in desk while offering you a beverage. \nWe had a one bedroom suite that accommodated four people reasonably well. Plenty of closet space, well lit with floor to ceiling windows, and actually quiet!\nThe bathroom was large with a very nice walk-in shower and a built-in bench with unique low spout to wash your feet.\nThe kitchenette was a nice touch with a stocked fridge offering complimentary non-alcoholic beverages and snacks, dishes and utensils, a sink, dishwasher, and a microwave. \nThey have daily Happy Hour(s) where you can get a complimentary decent glass of wine in the modest Lobby Lounge and bring it to your room. The Lobby Lounge has some seating and one table with 8 or so chairs where you can buy food from the adjacent restaurant. One suggestion is to offer selections of cheese and crackers platters to go with that wine. We ordered one that had to be custom made, but it worked well. \nWe didn't eat in the hotel restaurants. The restaurant off the lobby was very pricey for breakfast (+$20 per entree). When you can get a decent breakfast within a block or two of the hotel for under $10, what can I say?\nAs a hotel designer and someone who is very picky about hotels, I will definitely stay there again and I highly recommend it.",4.0,5.0,5.0,5.0,1762573
4,“Its the best of the Andaz Brand in the US....”,"I have stayed at each of the US Andaz properties, and this one is the best of them all. MUCH better than the West Hollywood property where Brand standards are slowly fading. This is the shining star here in the States.\nMy room (suite) on the 12th floor was fantastic as usual. Perfectly maintained, very comfortable, great bathrooms (best in NYC???). The bathroom is huge, with large shower 'room' and multiple shower heads. Double lavatories flank one wall. Huge....\nTHe bed is always perfect. Televisions in both rooms were perfect for my limited time spend in the suite. Nice desk for work area and in room dining. \nEvery room has a fridge with complimentary soft drinks, snacks, water, sparkling water, milk and juices. Nice touch. Suite has full size refrigerator with icemaker, and also features a microwave, sink and dishwasher drawer....all nicely concealed behind beautiful cabinetry. Really great for a longer stay...\nI do have 2 complaints. First, the elevators. The waits were sometimes ridiculous. 3 elevators serve the main tower with a rather limited number of rooms. THis is a newer hotel, but there is no service elevator. I was continually surprised after waiting for an elevator at how many stops we made for housekeeping to enter with carts, room service carts and trays, etc. How could Hyatt skip a service elevator on this new property? It really stinks at times.....its bad.\nLastly, i have traveled with world and certainly know NYC pricing. Breakfast at the Andaz ranks above almost anything i have ever paid. Apparently, i had a $190.00 breakfast for two the first morning, and $175.00 the second. Eggs for each, coffee, berries. Wow. Of course, i really don't know...as there is no bill. They just deliver it, and leave. Prices are listed in the in-s

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#tf is a vectorizer
#for normalized vector use tfid
tf = TfidfVectorizer(min_df=2, max_df=0.8, use_idf=False)
dtm = tf.fit_transform(lax["text"])

In [ ]:
#document term matrix 45584 non zero elements
dtm

<636x3377 sparse matrix of type '<class 'numpy.float64'>'
	with 63845 stored elements in Compressed Sparse Row format>

## Apply dimensionality reduction

The original formulation of Latent Semantic Indexing used SVD for dimensionality reduction. The sklearn function [`TruncatedSVD`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)  is an efficient implentation of SVD for large sparse matrices (such as dtm's). The argument `n_components` sets the number of dimensions (i.e., columns) in the reduced matrix.

You can experiment with other dimensionality reduction techniques, as described in the manual sections on [Matrix factorization](https://scikit-learn.org/stable/modules/decomposition.html#lsa) and [Manifold](https://scikit-learn.org/stable/modules/manifold.html#t-sne). Or, you can skip this step and use your dtm directly as input to the clusterer



In [ ]:
#truncatedsvd for large matrices
from sklearn.decomposition import TruncatedSVD

In [ ]:
#for restesting change random_state
lsi = TruncatedSVD(n_components=25, random_state=760)
dtm_lsi = lsi.fit_transform(dtm)

In [ ]:
dtm_lsi.shape

(636, 25)

In [ ]:
# Used PCA for dimensionality reduction

from sklearn.decomposition import PCA

#set number of components
n_components = 25
pca_model = PCA(n_components=n_components, random_state=760 )
pca_model.fit(dtm.toarray())
pca_dtm = pca_model.transform(dtm.toarray())









In [ ]:
pca_dtm.shape

(636, 25)

In [ ]:
# Used NMF for dimensionality reduction
# non negative matrix factorization method for dimensionality reduction
from sklearn.decomposition import NMF



In [ ]:
#set number of components (topics)
n_components = 25
nmf_model = NMF(n_components=n_components, random_state=760)
nmf_model.fit(dtm)
nmf_dtm = nmf_model.transform(dtm)



In [ ]:
nmf_dtm.shape

(636, 25)

## Perform cluster analysis

Sklearn has implementations of most current clusterining algoroths. See the [clustering chapter](https://scikit-learn.org/stable/modules/clustering.html#clustering) for more discussion. 

The [k-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) algorithm is a particular simple one. It requires one argument: `n_clusters` sets the number of clusters that k-means will find. K-means is a non-deterministic algorithm, meaning each time you run it you may get a different answer. If you want your results to be reproducible, set the `random_state` argument.




In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=15, random_state=858, n_init='auto')
cluster = kmeans.fit_predict(dtm_lsi)

In [ ]:
#hierarchical clustering 
#from sklearn.cluster import AgglomerativeClustering

# perform clustering
#agg_model = AgglomerativeClustering(n_clusters=15)
#agg_cluster = agg_model.fit_predict(dtm.toarray())

# print cluster labels
#print(agg_cluster)

## Interpret results

Now the hard part: what does all this mean? There are two ways of evaluating a clustering solution, quantitatively and qualitatively.

In [ ]:
from sklearn import metrics

A simple quantitative metric: how many documents are in each cluster?

In [ ]:
nltk.FreqDist(cluster).most_common()

[(7, 57),
 (14, 54),
 (6, 54),
 (4, 53),
 (1, 50),
 (3, 47),
 (8, 45),
 (12, 41),
 (0, 39),
 (9, 38),
 (10, 37),
 (2, 36),
 (11, 32),
 (5, 29),
 (13, 24)]

More complex metrics take into account how diffuse the clusters are and how well separated they are. Three potentially useful metrics are:

- [Calinski Harabasz_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html#sklearn.metrics.calinski_harabasz_score) [higher = better]
- [Davies Bouldin score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html#sklearn.metrics.davies_bouldin_score) [lower = better]
- [Silhouette_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score) [higher = better]

These are mostly useful for comparing different clusterings of the same data. For example, we could use these to find a good value for *k*, the number of clusters






In [ ]:
for k in range(2, 20):
    _kmeans = KMeans(n_clusters=k, random_state=858, n_init="auto")
    _cluster = _kmeans.fit_predict(dtm_lsi)
    print(
        f"{k:2}  "
        f"{metrics.calinski_harabasz_score(dtm_lsi, _cluster):7.3f}   "
        f"{metrics.davies_bouldin_score(dtm_lsi, _cluster):7.3f}   "
        f"{metrics.silhouette_score(dtm_lsi, _cluster):7.3f}  "
    )

 2   74.360     2.886     0.101  
 3   56.869     2.966     0.072  
 4   49.961     2.910     0.076  
 5   44.907     2.704     0.069  
 6   39.063     2.818     0.056  
 7   34.831     2.968     0.055  
 8   31.270     3.080     0.047  
 9   30.159     2.857     0.052  
10   27.124     2.975     0.045  
11   25.400     2.941     0.045  
12   23.794     2.996     0.044  
13   23.408     2.851     0.047  
14   22.195     2.848     0.045  
15   20.234     2.953     0.041  
16   19.798     2.860     0.042  
17   19.261     2.846     0.044  
18   18.420     2.796     0.043  
19   17.870     2.744     0.042  


Qualitative evaluation means reading the data and trying to figure it out. We could look at all of the reviews in some of the smaller clusters:

In [ ]:
lax[cluster == 5]["text"].head()

4       I have stayed at each of the US Andaz properties, and this one is the best of them all. MUCH better than the West Hollywood property where Brand standards are slowly fading. This is the shining star here in the States.\nMy room (suite) on the 12th floor was fantastic as usual. Perfectly maintained, very comfortable, great bathrooms (best in NYC???). The bathroom is huge, with large shower 'room' and multiple shower heads. Double lavatories flank one wall. Huge....\nTHe bed is always perfect. Televisions in both rooms were perfect for my limited time spend in the suite. Nice desk for work area and in room dining. \nEvery room has a fridge with complimentary soft drinks, snacks, water, sparkling water, milk and juices. Nice touch. Suite has full size refrigerator with icemaker, and also features a microwave, sink and dishwasher drawer....all nicely concealed behind beautiful cabinetry. Really great for a longer stay...\nI do have 2 complaints. First, the elevators. The waits were

Or, since we're using *k* means, we could sort them in order of proximity to the cluster centroid.

In [ ]:
#centriod 
C = 5
distances = metrics.pairwise.euclidean_distances(
    kmeans.cluster_centers_[C, np.newaxis, :], dtm_lsi[cluster == C, :]
)
lax[cluster == C].iloc[np.argsort(distances)[0]]["text"].head(5)

452                                                                                                                                     Great looking hotel with friendly staff and good location. Highlights include the well-designed bathrooms and showers, the automated window shades and the free snacks and non-alcoholic minibar items. Friendly but low key, not 'in your face' service -- and I mean that in a good way. Also, Andaz has free in-room wi-fi, which is a must. I will definitely choose this place again, and in fact have a current reservation for later this year. \nJust a few minor suggestions to get it from really good to perfect:\n--the baggage holding area is out in the open in a corner of the library/lounge area. This is unsightly and also lacks proper security for peoples' belongings. \n--the food at the restaurant is just OK. One dinner I had there was particularly substandard. However, in the hotel's defense, they offered to comp the meal without any unpleasantness whatsoev

## Term clustering

Document clustering is not usually very helpful, unless the documents are very short and/or naturally fall into a small number of distinct classes. Term clustering, on the other hand, does sometimes give interesting results. 

To cluster terms, all we need to do is transpose the document-term matrix to form the term-document matrix (tdm). After that, the rest of the process is the same, except that instead of relating documents by the terms they share, we're relating terms by the documents they co-occur in.



In [ ]:
tf = TfidfVectorizer(min_df=2, max_df=0.5, use_idf=True)
tdm = tf.fit_transform(lax["text"]).T

lsi = TruncatedSVD(n_components=5, random_state=858)
tdm_lsi = lsi.fit_transform(tdm)

kmeans = KMeans(15, random_state=858, n_init="auto")
cluster = kmeans.fit_predict(tdm_lsi)

In [ ]:
nltk.FreqDist(cluster).most_common()

[(6, 2117),
 (0, 621),
 (8, 198),
 (4, 99),
 (12, 99),
 (13, 60),
 (14, 33),
 (1, 32),
 (9, 23),
 (11, 22),
 (10, 21),
 (5, 12),
 (2, 9),
 (3, 5),
 (7, 2)]

In [ ]:
vocab = tf.get_feature_names_out()

In [ ]:
vocab[cluster == 5]

array(['alcoholic', 'bar', 'complimentary', 'drinks', 'free', 'lobby',
       'mini', 'nice', 'non', 'shower', 'snacks', 'wine'], dtype=object)

In [ ]:
C = 5
distances = metrics.pairwise.euclidean_distances(
    kmeans.cluster_centers_[C, np.newaxis, :], tdm_lsi[cluster == C, :]
)
vocab[cluster == C][np.argsort(distances)[0]]

array(['complimentary', 'wine', 'drinks', 'snacks', 'mini', 'non',
       'lobby', 'alcoholic', 'bar', 'shower', 'nice', 'free'],
      dtype=object)

## Results

Once you're done experimenting with document and term clustering, write up a paragraph on your results? What methods did you try? What worked and what didn't? Did you manage to find any useful information about hotels from this?

The above code performs cluster analysis on hotel reviews to group similar documents and terms together.In the first step, a document term matrix (DTM) was constructed using the TfidVectorizer which creates a matrix where the values represents the frequency of a term in a document. Later,dimension reduction was performed using three techniques: TruncatedSVD, PCA(Principal component analysis) and NMF(Non-negative matrix factorization) and they all reduce the dimensionality of the document-term matrix to 25 dimensions. Now, Cluster Analysis was performed using K means algorithm to group the reviews into 15 clusters. The most common clusters were labeled 7,14,6,4,1 respectively.To interpret the results, the most frequent terms in each cluster were examined.The results are interpreted using both qualitative and quantitative methods.Here I have taken cluster 5 which displays the as an example of this qualitatitive evaluation.The results show the text of four reviews that praise the Andaz hotel in Manhattan for its location, design, service, and amenities, including a complimentary mini-bar and well-designed bathrooms with large showers. Some minor complaints include long wait times for elevators, an unsightly baggage holding area, and mediocre restaurant food. Overall, the reviews indicate a positive guest experience and a desire to return to the hotel. The following code is sorting the reviews for a particular cluster (cluster 5) in order of their proximity to the centroid of that cluster.  The resulting output shows the top 5 reviews that are closest to the centroid of cluster 5, with the corresponding review text. These reviews generally speak positively about the hotel's location, service, and amenities.

In term clustering, the most common clusters are identified and the terms belonging to a particular cluster (cluster 5) are displayed.The results show that the terms in cluster 5 are related to complimentary alcoholic drinks, snacks, and mini bar amenities that are provided in a hotel's lobby. The top terms representative of this cluster are "complimentary", "wine", "drinks", "snacks", "mini", and "lobby"